In [3]:
import mlxops_sdk_function
from backend_central_dev.constant import TaskStatus, TaskExecution, PipelineExecution, PipelineSheet
central_url = "http://127.0.0.1:5006"

In [7]:
# pipeline_name = "dummy_training_pipeline"
pipeline_name = "dummy_training_xai_pipeline"
num_exp = 97
with tqdm(total=num_exp) as pbar:
    for i in range(num_exp):
        new_pipeline_execution = mlxops_sdk_function.sheet_operation_request(
            central_url,
            sheet_type='pipeline_sheet',
            sheet_name=pipeline_name,
            operation="execute"
        )

        def get_last_execution():
            return mlxops_sdk_function.get_execution_list(
                central_url,
                execution_type='pipeline_execution',
                sheet_name=pipeline_name,
            )[-1]
        # time.sleep(0.5)
        last_pipeline_execution = get_last_execution()
        assert last_pipeline_execution[PipelineExecution.pipeline_ticket] == new_pipeline_execution[PipelineExecution.pipeline_ticket]
        while last_pipeline_execution[TaskExecution.task_status] != TaskStatus.finished:
            last_pipeline_execution = get_last_execution()
            time.sleep(3)
            pbar.set_postfix({
                "status": last_pipeline_execution[TaskExecution.task_status],
                "pipeline": last_pipeline_execution[PipelineExecution.pipeline_execution_name]
            })

        pbar.update(1)

  0%|          | 0/97 [00:00<?, ?it/s]

In [8]:
task_execution_type_list = [
    'dummy_training_task',
    'dummy_model_evaluation_task',
    'dummy_xai_task',
    'dummy_xai_evaluation_task'
]

rs_data = []
for task_sheet_name in task_execution_type_list:
    executions = mlxops_sdk_function.get_execution_list(
        central_url,
        execution_type="task_execution",
        sheet_name=task_sheet_name
    )
    # print(f"Total {task_sheet_name} executions: {len(executions)}")
    time_costs = []
    for execution in executions:
        if (type(execution['start_time']) is not float) or (type(execution['end_time']) is not float):
            continue
        time_cost = execution['end_time'] - execution['start_time']
        time_costs.append(time_cost)

    rs_data.append({
        "task_sheet_name": task_sheet_name,
        "time_costs": np.array(time_costs).mean(),
        "task_num": len(executions)
    })

pd.DataFrame(rs_data)

,task_sheet_name,time_costs,task_num
0,dummy_training_task,3.722807,100
1,dummy_model_evaluation_task,3.272411,100
2,dummy_xai_task,3.245595,100
3,dummy_xai_evaluation_task,3.253295,100
